In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from  sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import StratifiedKFold
import itertools

from sklearn.feature_extraction.text import CountVectorizer
le = LabelEncoder() 
import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Loading DataSet
>load training dataset to train model and testing dataset to evaluate model
>I split dataset into two file traing set and testing set

In [ ]:
#Read train data
train= pd.read_csv('/content/drive/MyDrive/ML task/ML/train1.csv',encoding='utf-8')
train.dialect.value_counts()

EG    1556
PL    1181
KW    1137
LY     985
QA     839
JO     754
LB     746
SA     724
AE     710
BH     710
OM     517
SY     438
DZ     437
IQ     419
SD     390
MA     311
YE     268
TN     249
Name: dialect, dtype: int64

In [ ]:
len(train)

12371

In [ ]:
train.head()

,id,text,dialect
0,1.040000e+18,مافش فرق بينا وبين القديد المخلل,LY
1,1.110000e+18,الجبرين معتوه ثور عطوه الهلال,TN
2,1.170000e+18,طحن الماكينات المانيا هولندا,AE
3,6.440000e+17,تو مافي,OM
4,1.160000e+18,العراقي وكح وين يروح شوف شسوة بسعودييون لندن,IQ


In [ ]:
train=train.dropna()

In [ ]:
#train=train[0:2000]

In [ ]:
y_train = train.dialect                           
X_train= train.drop('dialect',axis = 1 )


In [ ]:
test= pd.read_csv("/content/drive/MyDrive/ML task/ML/unseen1.csv")
test=test.dropna()
y_test = test.dialect                           
X_test= test.drop('dialect',axis = 1 )

print(test['dialect'].value_counts())

EG    173
PL    131
KW    126
LY    110
QA     93
JO     84
LB     83
SA     81
BH     79
AE     79
OM     57
SY     49
DZ     48
IQ     46
SD     43
MA     35
YE     30
TN     28
Name: dialect, dtype: int64


In [ ]:


# Set values for various parameters of word2vec
num_features = 500 # Word vector dimensionality. Determines the no of words each word in the vocabulary will
#be associated with. Must be tuned.
min_word_count = 50   # Minimum word count. Wods occuring below the threshold will be ignored
num_workers = 1       # Number of threads to run in parallel
context = 15          # Context window size to be considered for each word                                             
downsampling = 1e-3   # Downsample setting for frequent words. To prevent more frequent words from dominating.



In [ ]:
from gensim.models import word2vec

#Model Word2Vec
model = word2vec.Word2Vec(train['text'], workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If the model is not going to be trained further, init_sims can be called 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [ ]:
from tqdm import tqdm
def createFeatureVector(words, model, num_features):
    #initialize a 1D array with length as num of features of word2vec model chosen by us. 
    #Here it is 250.
    featVector = np.zeros((num_features,),dtype="float32")
    
    nWords = 0
    
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, as set is faster
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word and add it to the feature vector to get the total sum of feature vectors of the
    #entire review
    for word in words:
        if word in index2word_set: 
            nWords = nWords + 1.
            featVector = np.add(featVector,model[word])
            
    # Divide the result by the number of words to get the average of the feature vectors of 
    #all words in the review
    if(nWords != 0):
        featVector = np.divide(featVector,nWords)
    return featVector

#calculates the average of the feature vectors for each review using the word2vec values assigned for 
#each word
def avgFeatureVectors(sentences, model, num_features):
    overallFeatureVectors = []
    for sentence in tqdm(sentences):
        overallFeatureVectors.append(createFeatureVector(sentence, model, num_features)) 
    return overallFeatureVectors

train_vect = avgFeatureVectors( train['text'], model, num_features )
test_vect = avgFeatureVectors( test['text'] ,model, num_features )

100%|██████████| 1375/1375 [00:00<00:00, 3372.31it/s]


In [ ]:
#print result
def printResult(y_real, y_pred):
    acc = accuracy_score(y_real, y_pred)
    print("Accuracy: {:.2f}".format(acc*100),end='\n\n')
    print(classification_report(y_real,y_pred))


## Classifiers
## Model & Training & Evaluation & Result

## **KNN **

In [ ]:
KNN= KNeighborsClassifier()
model_KNN=KNN.fit(train_vect,y_train)


In [ ]:
#Save KNN model
import joblib

joblib.dump(model_KNN, "/content/drive/MyDrive/ML task/ML/word2vec/KNN.pkl")
#model_KNN.save("/content/drive/MyDrive/ML task/ML/ML_Model_using_Count_Vector/KNN.h5")

['/content/drive/MyDrive/ML task/ML/word2vec/KNN.pkl']

In [ ]:
y_train_pred=model_KNN.predict(train_vect)
print("Train Result of KNN")
printResult(y_train,y_train_pred)

Train Result of KNN
Accuracy: 34.12

              precision    recall  f1-score   support

          AE       0.25      0.67      0.36       709
          BH       0.29      0.57      0.39       710
          DZ       0.24      0.46      0.32       436
          EG       0.35      0.62      0.45      1556
          IQ       0.30      0.25      0.27       416
          JO       0.34      0.28      0.30       754
          KW       0.38      0.36      0.37      1137
          LB       0.39      0.30      0.34       746
          LY       0.42      0.28      0.34       983
          MA       0.47      0.18      0.26       311
          OM       0.47      0.16      0.24       517
          PL       0.46      0.28      0.35      1181
          QA       0.41      0.25      0.31       838
          SA       0.45      0.21      0.29       724
          SD       0.39      0.09      0.15       390
          SY       0.41      0.10      0.16       436
          TN       0.46      0.08      0.13 

In [ ]:
y_test_pred=model_KNN.predict(test_vect)
print("Test Result of KNN")
printResult(y_test,y_test_pred)

Test Result of KNN
Accuracy: 9.75

              precision    recall  f1-score   support

          AE       0.06      0.18      0.09        79
          BH       0.05      0.08      0.06        79
          DZ       0.06      0.12      0.08        48
          EG       0.16      0.32      0.22       173
          IQ       0.07      0.07      0.07        46
          JO       0.06      0.05      0.05        84
          KW       0.16      0.13      0.14       126
          LB       0.06      0.04      0.04        83
          LY       0.04      0.03      0.03       110
          MA       0.06      0.03      0.04        35
          OM       0.05      0.02      0.03        57
          PL       0.12      0.08      0.09       131
          QA       0.09      0.06      0.07        93
          SA       0.07      0.04      0.05        81
          SD       0.09      0.02      0.04        43
          SY       0.22      0.04      0.07        49
          TN       0.00      0.00      0.00   

**Random forest **


In [ ]:
RF=RandomForestClassifier()
model_RF=RF.fit(train_vect,y_train)
joblib.dump(model_RF, "/content/drive/MyDrive/ML task/ML/word2vec/RF.pkl")

['/content/drive/MyDrive/ML task/ML/word2vec/RF.pkl']

In [ ]:
y_train_pred=model_RF.predict(train_vect)
print("Train Result of Random Forest")
printResult(y_train,y_train_pred)

Train Result of Random Forest
Accuracy: 99.99

              precision    recall  f1-score   support

          AE       1.00      1.00      1.00       709
          BH       1.00      1.00      1.00       710
          DZ       1.00      1.00      1.00       436
          EG       1.00      1.00      1.00      1556
          IQ       1.00      1.00      1.00       416
          JO       1.00      1.00      1.00       754
          KW       1.00      1.00      1.00      1137
          LB       1.00      1.00      1.00       746
          LY       1.00      1.00      1.00       983
          MA       1.00      1.00      1.00       311
          OM       1.00      1.00      1.00       517
          PL       1.00      1.00      1.00      1181
          QA       1.00      1.00      1.00       838
          SA       1.00      1.00      1.00       724
          SD       1.00      1.00      1.00       390
          SY       1.00      1.00      1.00       436
          TN       1.00      1.00 

In [ ]:
y_test_pred=model_RF.predict(test_vect)
print("Test Result of Random Forest")
printResult(y_test,y_test_pred)

Test Result of Random Forest
Accuracy: 12.73

              precision    recall  f1-score   support

          AE       0.06      0.03      0.04        79
          BH       0.06      0.03      0.03        79
          DZ       0.12      0.02      0.04        48
          EG       0.16      0.50      0.24       173
          IQ       0.00      0.00      0.00        46
          JO       0.03      0.01      0.02        84
          KW       0.15      0.25      0.18       126
          LB       0.09      0.10      0.09        83
          LY       0.06      0.05      0.06       110
          MA       0.00      0.00      0.00        35
          OM       0.17      0.02      0.03        57
          PL       0.14      0.20      0.16       131
          QA       0.11      0.09      0.10        93
          SA       0.06      0.04      0.05        81
          SD       0.00      0.00      0.00        43
          SY       0.00      0.00      0.00        49
          TN       0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***Decision tree ***

In [ ]:
DT=DecisionTreeClassifier()
model_DT=DT.fit(train_vect, y_train)
joblib.dump(model_DT, "/content/drive/MyDrive/ML task/ML/word2vec/DT.pkl")

['/content/drive/MyDrive/ML task/ML/word2vec/DT.pkl']

In [ ]:
y_train_pred=model_DT.predict(train_vect)
print("Train Result of DT")
printResult(y_train,y_train_pred)

Train Result of DT
Accuracy: 99.99

              precision    recall  f1-score   support

          AE       1.00      1.00      1.00       709
          BH       1.00      1.00      1.00       710
          DZ       1.00      1.00      1.00       436
          EG       1.00      1.00      1.00      1556
          IQ       1.00      1.00      1.00       416
          JO       1.00      1.00      1.00       754
          KW       1.00      1.00      1.00      1137
          LB       1.00      1.00      1.00       746
          LY       1.00      1.00      1.00       983
          MA       1.00      1.00      1.00       311
          OM       1.00      1.00      1.00       517
          PL       1.00      1.00      1.00      1181
          QA       1.00      1.00      1.00       838
          SA       1.00      1.00      1.00       724
          SD       1.00      1.00      1.00       390
          SY       1.00      1.00      1.00       436
          TN       1.00      1.00      1.00  

In [ ]:
y_test_pred=model_DT.predict(test_vect)
print("Test Result of DT")
printResult(y_test,y_test_pred)

Test Result of DT
Accuracy: 7.71

              precision    recall  f1-score   support

          AE       0.04      0.04      0.04        79
          BH       0.04      0.04      0.04        79
          DZ       0.04      0.04      0.04        48
          EG       0.15      0.13      0.14       173
          IQ       0.04      0.04      0.04        46
          JO       0.05      0.05      0.05        84
          KW       0.10      0.10      0.10       126
          LB       0.08      0.07      0.08        83
          LY       0.04      0.05      0.04       110
          MA       0.03      0.03      0.03        35
          OM       0.05      0.05      0.05        57
          PL       0.10      0.08      0.08       131
          QA       0.12      0.16      0.14        93
          SA       0.13      0.15      0.14        81
          SD       0.02      0.02      0.02        43
          SY       0.05      0.04      0.04        49
          TN       0.00      0.00      0.00    



**Support Vector Machine**


In [ ]:
SVM=SVC()
model_SVM=SVM.fit(train_vect, y_train)
joblib.dump(model_SVM, "/content/drive/MyDrive/ML task/ML/word2vec/SVM.pkl")

['/content/drive/MyDrive/ML task/ML/word2vec/SVM.pkl']

In [ ]:
y_train_pred=model_DT.predict(train_vect)
print("Train Result of SVM")
printResult(y_train,y_train_pred)

Train Result of SVM
Accuracy: 99.99

              precision    recall  f1-score   support

          AE       1.00      1.00      1.00       709
          BH       1.00      1.00      1.00       710
          DZ       1.00      1.00      1.00       436
          EG       1.00      1.00      1.00      1556
          IQ       1.00      1.00      1.00       416
          JO       1.00      1.00      1.00       754
          KW       1.00      1.00      1.00      1137
          LB       1.00      1.00      1.00       746
          LY       1.00      1.00      1.00       983
          MA       1.00      1.00      1.00       311
          OM       1.00      1.00      1.00       517
          PL       1.00      1.00      1.00      1181
          QA       1.00      1.00      1.00       838
          SA       1.00      1.00      1.00       724
          SD       1.00      1.00      1.00       390
          SY       1.00      1.00      1.00       436
          TN       1.00      1.00      1.00 

In [ ]:
y_test_pred=model_DT.predict(test_vect)
print("Test Result of SVM")
printResult(y_test,y_test_pred)

Test Result of SVM
Accuracy: 7.71

              precision    recall  f1-score   support

          AE       0.04      0.04      0.04        79
          BH       0.04      0.04      0.04        79
          DZ       0.04      0.04      0.04        48
          EG       0.15      0.13      0.14       173
          IQ       0.04      0.04      0.04        46
          JO       0.05      0.05      0.05        84
          KW       0.10      0.10      0.10       126
          LB       0.08      0.07      0.08        83
          LY       0.04      0.05      0.04       110
          MA       0.03      0.03      0.03        35
          OM       0.05      0.05      0.05        57
          PL       0.10      0.08      0.08       131
          QA       0.12      0.16      0.14        93
          SA       0.13      0.15      0.14        81
          SD       0.02      0.02      0.02        43
          SY       0.05      0.04      0.04        49
          TN       0.00      0.00      0.00   


**Logistic Regrssion *italicized text*




In [ ]:
LR= LogisticRegression()

model_LR=LR.fit(train_vect, y_train)
joblib.dump(model_LR, "/content/drive/MyDrive/ML task/ML/word2vec/LR.pkl")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


['/content/drive/MyDrive/ML task/ML/word2vec/LR.pkl']

In [ ]:
y_train_pred=model_LR.predict(train_vect)
print("Train Result of LR")
printResult(y_train,y_train_pred)

Train Result of LR
Accuracy: 13.44

              precision    recall  f1-score   support

          AE       0.00      0.00      0.00       709
          BH       0.12      0.00      0.01       710
          DZ       0.00      0.00      0.00       436
          EG       0.13      0.90      0.23      1556
          IQ       0.00      0.00      0.00       416
          JO       0.00      0.00      0.00       754
          KW       0.12      0.09      0.10      1137
          LB       0.15      0.04      0.06       746
          LY       0.00      0.00      0.00       983
          MA       0.00      0.00      0.00       311
          OM       0.00      0.00      0.00       517
          PL       0.14      0.11      0.12      1181
          QA       0.00      0.00      0.00       838
          SA       0.17      0.02      0.03       724
          SD       0.00      0.00      0.00       390
          SY       0.00      0.00      0.00       436
          TN       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_test_pred=model_LR.predict(test_vect)
print("Test Result of LR")
printResult(y_test,y_test_pred)

Test Result of LR
Accuracy: 13.24

              precision    recall  f1-score   support

          AE       0.00      0.00      0.00        79
          BH       0.00      0.00      0.00        79
          DZ       0.00      0.00      0.00        48
          EG       0.14      0.93      0.24       173
          IQ       0.00      0.00      0.00        46
          JO       0.00      0.00      0.00        84
          KW       0.09      0.06      0.07       126
          LB       0.07      0.01      0.02        83
          LY       0.00      0.00      0.00       110
          MA       0.00      0.00      0.00        35
          OM       0.00      0.00      0.00        57
          PL       0.13      0.08      0.10       131
          QA       0.00      0.00      0.00        93
          SA       0.18      0.02      0.04        81
          SD       0.00      0.00      0.00        43
          SY       0.00      0.00      0.00        49
          TN       0.00      0.00      0.00   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
